# Amazon Personalize: using text as unstructured item metadata

The relevance of the recommendations you deliver with Amazon Personalize depends on the data available when the recommendations are generated. Amazon Personalize uses your users’ historical interactions, the attributes of your items, and your users’ metadata to learn what items are most relevant for each user. The primary data required by Amazon Personalize is user-item interactions. The interactions users have with items in your catalog, such as clicking on a product, reading an article, watching a video, or purchasing a product, are an important signal of what they have found relevant in the past. Including item and user attributes, also known as metadata, can enhance the relevance of recommendations; especially for new items that are similar to what your users have found relevant. However, structured metadata such as an item’s category, style, or genre may not always be readily available or doesn’t provide all the information that you have in your narrative descriptions. Now Amazon Personalize allows you to add unstructured metadata such as product descriptions, video transcripts, or article text with your other item attributes. Amazon Personalize hosts, manages, and automatically uses natural language processing (NLP) models to process your text and use it to improve the performance of your Amazon Personalize solutions.

This notebook will demonstrate how text in the form of product descriptions can be included as unstructured item metadata to improve the relevance of recommendations.

Amazon Reviews data from the Amazon Prime Pantry category are used for the interactions and items datasets.

When considering including text in your items dataset, keep the following best practices in mind.
- Text that is editorially validated to be concise, relevant, and informative to each item, where the most pertinent details are mentioned earlier in the text, is preferred over user generated content that may be less relevant or consistent
- A sparsely populated text column will diminish the positive impact of including text in the items dataset
- Cleanse all text of markup and superfluous whitespace formatting before adding to it to the items dataset
- English is currently the only supported language for the text field
- Text fields are currently only considered for the User-Personalization and Personalized-Ranking recipes

Two dataset groups will be created that will include data with and without item descriptions so we can train separate models and compare their offline and online results.

In [1]:
import pandas as pd
import json
import numpy as np
from datetime import datetime
import boto3
import time
from time import sleep
from lxml import html

## Load and inspect datasets

We'll start by loading the Prime Pantry reviews dataset. You will need to fill out the form for access to the data files:

http://deepyeti.ucsd.edu/jianmo/amazon/index.html

Citation:
> Justifying recommendations using distantly-labeled reviews and fined-grained aspects  
> Jianmo Ni, Jiacheng Li, Julian McAuley  
> Empirical Methods in Natural Language Processing (EMNLP), 2019 [pdf](http://cseweb.ucsd.edu/~jmcauley/pdfs/emnlp19a.pdf)

In [2]:
data_dir = 'raw_data'
!mkdir $data_dir

!cd $data_dir && \
    wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Prime_Pantry.json.gz && \
    wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Prime_Pantry.json.gz

mkdir: cannot create directory ‘raw_data’: File exists
--2021-07-13 22:06:52--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Prime_Pantry.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45435146 (43M) [application/octet-stream]
Saving to: ‘Prime_Pantry.json.gz’

Prime_Pantry.json.g 100%[===================>]  43.33M  12.3MB/s    in 4.8s    

2021-07-13 22:06:57 (9.01 MB/s) - ‘Prime_Pantry.json.gz’ saved [45435146/45435146]

--2021-07-13 22:06:57--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Prime_Pantry.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5281662 (5.0M) [application/octet-stream]
Saving to: ‘meta_Prime_Pantry.json.gz’

meta_Prime_Pa

### Load and inspect reviews data

We'll start by loading the reviews dataset for the Prime Pantry products and running some commands to see what we have to work with.

In [3]:
pantry_df = pd.read_json(data_dir + '/Prime_Pantry.json.gz', lines=True, compression='infer')
pantry_df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style
0,5,True,"12 14, 2014",A1NKJW0TNRVS7O,B0000DIWNZ,Tamara M.,Good clinging,Clings well,1418515200,NaN,NaN,NaN
1,4,True,"11 20, 2014",A2L6X37E8TFTCC,B0000DIWNZ,Amazon Customer,Fantastic buy and a good plastic wrap. Even t...,Saran could use more Plus to Cling better.,1416441600,NaN,NaN,NaN
2,4,True,"10 11, 2014",A2WPR4W6V48121,B0000DIWNZ,noname,ok,Four Stars,1412985600,NaN,NaN,NaN
3,3,False,"09 1, 2014",A27EE7X7L29UMU,B0000DIWNZ,ZapNZs,Saran Cling Plus is kind of like most of the C...,"The wrap is fantastic, but the dispensing, cut...",1409529600,4,NaN,NaN
4,4,True,"08 10, 2014",A1OWT4YZGB5GV9,B0000DIWNZ,Amy Rogers,This is my go to plastic wrap so there isn't m...,has been doing it's job for years,1407628800,NaN,NaN,NaN


In [4]:
pantry_df.shape

(471614, 12)

What can we learn from this output? There are over 471K reviews and 12 columns of data. The `asin` column is our unique item identifier, `reviewerID` is our unique user identifier, `unixReviewTime` is our timestamp for the review, and `overall` indicates the positivity of the review on a scale of 1-5. We will use this file as the basis for our interactions dataset for Personalize. 

### Build and save interactions dataset

Let's start building our interactions dataset by narrowing down the rows we want to include. The first step is to isolate only the positive reviews. For this we will assume any reviews with an overall rating of 4 or higher is a positive review. Anything rating of 3 or below are either mediocre or negative reviews.

In [5]:
positive_reviews_df = pantry_df[pantry_df['overall'] > 3]
positive_reviews_df.shape

(387692, 12)

We're down to 387K positive reviews. Still plenty for training a model in Personalize.

Next let's narrow down the dataset to just the columns we need and add an `EVENT_TYPE` column to indicate the type of events we're capturing. Adding an `EVENT_TYPE` column now will make it easier to explore testing real-time events later if you choose to do so (since `eventType` is a required field for the [PutEvents](https://docs.aws.amazon.com/personalize/latest/dg/API_UBS_PutEvents.html) API).

In [6]:
positive_reviews_df = positive_reviews_df[['reviewerID', 'asin', 'unixReviewTime', 'overall']]
positive_reviews_df['EVENT_TYPE']='reviewed'

positive_reviews_df.head()

,reviewerID,asin,unixReviewTime,overall,EVENT_TYPE
0,A1NKJW0TNRVS7O,B0000DIWNZ,1418515200,5,reviewed
1,A2L6X37E8TFTCC,B0000DIWNZ,1416441600,4,reviewed
2,A2WPR4W6V48121,B0000DIWNZ,1412985600,4,reviewed
4,A1OWT4YZGB5GV9,B0000DIWNZ,1407628800,4,reviewed
5,A1GN2ADKF1IE7K,B0000DIWNZ,1405296000,5,reviewed


One last check we should do is sanity check a `unixReviewTime` column value. Since Personalize builds sequence models based on the date and time of each interaction, it's important that the timestamp of each interaction is represented in the expected format so that it is interpreted correctly.

Let's pick a value for the `unixReviewTime` column and parse it into a human-readable date so we can verify that it's reasonable.

In [7]:
time_stamp = positive_reviews_df.iloc[50]['unixReviewTime']
print(time_stamp)
print(datetime.utcfromtimestamp(time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

1321488000
2011-11-17 00:00:00


The timestamp value looks good. Let's get some final summary information for our dataset.

In [8]:
positive_reviews_df.describe(include='all')

,reviewerID,asin,unixReviewTime,overall,EVENT_TYPE
count,387692,387692,3.876920e+05,387692.000000,387692
unique,202254,10584,NaN,NaN,1
top,A35Q0RBM3YNQNF,B00XA9DADC,NaN,NaN,reviewed
freq,176,5288,NaN,NaN,387692
mean,NaN,NaN,1.468847e+09,4.847227,NaN
std,NaN,NaN,4.314975e+07,0.359769,NaN
min,NaN,NaN,1.073693e+09,4.000000,NaN
25%,NaN,NaN,1.447200e+09,5.000000,NaN
50%,NaN,NaN,1.474718e+09,5.000000,NaN
75%,NaN,NaN,1.498435e+09,5.000000,NaN


We have 387K reviews for 202K distinct reviewers/users across 10K unique products. This is basis of our interactions dataset.

Before we can use this as our interactions dataset, though, we need to rename the columns to match those expected by Personalize.

In [9]:
positive_reviews_df.rename(columns = {'reviewerID':'USER_ID', 'asin':'ITEM_ID', 
                              'unixReviewTime':'TIMESTAMP', 'overall': 'EVENT_VALUE'}, inplace = True)
positive_reviews_df

,USER_ID,ITEM_ID,TIMESTAMP,EVENT_VALUE,EVENT_TYPE
0,A1NKJW0TNRVS7O,B0000DIWNZ,1418515200,5,reviewed
1,A2L6X37E8TFTCC,B0000DIWNZ,1416441600,4,reviewed
2,A2WPR4W6V48121,B0000DIWNZ,1412985600,4,reviewed
4,A1OWT4YZGB5GV9,B0000DIWNZ,1407628800,4,reviewed
5,A1GN2ADKF1IE7K,B0000DIWNZ,1405296000,5,reviewed
...,...,...,...,...,...
471609,A19GSVHXVT5NNF,B01HI8JVI8,1494892800,5,reviewed
471610,ABSCTKLX9F9IU,B01HI8JVI8,1493769600,5,reviewed
471611,A2R33RCWKDHZ3L,B01HI8JVI8,1492646400,5,reviewed
471612,A2INGHYEXZDHMC,B01HI8JVI8,1492560000,5,reviewed


Finally, let's save our positive reviews dataframe as a CSV. We'll upload this CSV to Personalize later in this notebook.

In [10]:
interactions_filename = "interactions.csv"
positive_reviews_df.to_csv(interactions_filename, index=False, float_format='%.0f')

### Load and inspect item metadata

Now that we have the interactions dataset established, let's turn to the items dataset. This is where we will find the unstructured text value that we will include in the model.

Like the reviews dataset, the Prime Pantry item metadata file is also represented in JSON. Due to the nested nature of this file, this will present some challenges in getting our data formatted the way we need it.

Let's start by loading the metadata file into a dataframe and taking a look the data.

In [11]:
pantry_meta_df = pd.read_json('raw_data/meta_Prime_Pantry.json.gz', lines=True, compression='infer')
pantry_meta_df

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Sink your sweet tooth into MILK DUDS Candya d...,,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",,Milk Duds,[],[],[],"{'ASIN: ': 'B00005BPJO', 'Item model number:':...","<img src=""https://m.media-amazon.com/images/G/...",,NaT,$5.00,B00005BPJO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
1,[],,[Sink your sweet tooth into MILK DUDS Candya d...,,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",,Milk Duds,[],[],[],"{'ASIN: ': 'B00005BPJO', 'Item model number:':...","<img src=""https://m.media-amazon.com/images/G/...",,NaT,$5.00,B00005BPJO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
2,[],,[A perfect Lentil soup starts with Goya Lentil...,,"Goya Dry Lentils, 16 oz","[B003SI144W, B000VDRKEK]",,Goya,[],[],"[B074MFVZG7, B079PTH69L, B000VDRKEK, B074M9T81...",{'ASIN: ': 'B0000DIF38'},"<img src=""https://images-na.ssl-images-amazon....",,NaT,,B0000DIF38,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
3,[],,[Saran Premium Wrap is an extra tough yet easy...,,"Saran Premium Plastic Wrap, 100 Sq Ft","[B01MY5FHT6, B000PYF8VM, B000SRMDFA, B07CX6LN8...",,Saran,[],[],"[B077QLSLRQ, B00JPKW1RQ, B000FE2IK6, B00XUJHJ9...",{'Domestic Shipping: ': 'This item can only be...,"<img src=""https://images-na.ssl-images-amazon....",,NaT,,B0000DIWNI,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Eas...,,"Saran Cling Plus Plastic Wrap, 200 Sq Ft",[],,Saran,[],[],[B0014CZ0TE],{'Domestic Shipping: ': 'This item can only be...,"<img src=""https://images-na.ssl-images-amazon....",,NaT,,B0000DIWNZ,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10808,[],,[These bars are where our journey started and ...,,"KIND Bars, Caramel Almond &amp; Sea Salt, Glut...",[],,KIND,[],"26,259 in Grocery & Gourmet Food (","[B00JQQAN60, B00JQQAWSY, B0111K7V54, B0111K8L9...","{'ASIN: ': 'B01HI76312', 'Item model number:':...","<img src=""https://images-na.ssl-images-amazon....",,NaT,$3.98,B01HI76312,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
10809,[],,[These bars are where our journey started and ...,,"KIND Bars, Maple Glazed Pecan &amp; Sea Salt, ...",[],,KIND,[],"16,822 in Grocery & Gourmet Food (","[B0111K97JC, B00JQQAN60, B0111K8L9Y, B01HI7631...",{'ASIN: ': 'B01HI76790'},"<img src=""https://images-na.ssl-images-amazon....",,NaT,$5.81,B01HI76790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
10810,[],,[These bars are where our journey started and ...,,"KIND Bars, Dark Chocolate Almond &amp; Coconut...",[],,KIND,[],"107,057 in Grocery & Gourmet Food (","[B0111K7V54, B01HI76312, B00JQQAL0S, B0111K97J...",{'ASIN: ': 'B01HI76SA8'},"<img src=""https://images-na.ssl-images-amazon....",,NaT,$4.98,B01HI76SA8,[],[]
10811,[],,[These bars are where our journey started and ...,,"KIND Bars, Honey Roasted Nuts &amp; Sea Salt, ...",[],,KIND,[],"24,648 in Grocery & Gourmet Food (","[B00JQQAN60, B0111K7V54, B01HI76312, B0111K97J...",{'ASIN: ': 'B01HI76XS0'},"<img src=""https://images-na.ssl-images-amazon....",,NaT,$5.81,B01HI76XS0,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [12]:
pantry_meta_df.describe()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
count,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,0,10813,10813,10813,10813
unique,1,1,9409,1,10782,3957,1,1960,763,4828,5940,10786,4,1,0,1482,10812,8940,8940
top,[],,[],,"Infants' Motrin Concentrated Drops, Fever Redu...",[],,L'Oreal Paris,[],[],[],{},"<img src=""https://images-na.ssl-images-amazon....",,NaN,,B00005BPJO,[],[]
freq,10813,10813,98,10813,2,6754,10813,171,9777,5937,4835,24,10621,10813,NaN,4063,2,1781,1781


So what can we learn from this information? First, there are over 10K products represented in the metadata file. Most of the columns will be of little value to us for Personalize since they aren't relevant as features (image URLs, `details`, `also_viewed`, `also_buy`, etc) or are mostly blank/sparse (`category`, `fit`, `tech1`, etc). The `asin` column is our unique identifier for each item (although there looks to be one duplicate) and `brand` and `price` look like they may be useful. The `description` column is what we will use for unstructured text.

However, we have to do some cleanup and reformatting of the fields we want to use in our items dataset. For example, the `price` field is a formatted currency value (string) and not numeric and the `description` field was loaded as an array of strings due to how they values were represented and parsed from the original JSON file. Lastly, the `description` values also contain HTML markup that needs to stripped.

Let's start by creating a dataframe with just the columns we need for the items dataset.

In [13]:
items_df = pantry_meta_df.copy()
items_df = items_df[['asin', 'brand', 'price', 'description']]
items_df.head(10)

,asin,brand,price,description
0,B00005BPJO,Milk Duds,$5.00,[Sink your sweet tooth into MILK DUDS Candya d...
1,B00005BPJO,Milk Duds,$5.00,[Sink your sweet tooth into MILK DUDS Candya d...
2,B0000DIF38,Goya,,[A perfect Lentil soup starts with Goya Lentil...
3,B0000DIWNI,Saran,,[Saran Premium Wrap is an extra tough yet easy...
4,B0000DIWNZ,Saran,,[200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Eas...
5,B0000GH6UG,Ibarra,,"[Ibarra Chocolate, 19 Oz, , ]"
6,B0000KC2BK,Knorr,$3.09,[Knorr Granulated Chicken Flavor Bouillon is a...
7,B0001E1IN8,Castillo,,[Red chili habanero sauces. They are present t...
8,B00032E8XK,Chicken of the Sea,$1.48,[Chicken of the Sea Solid White Albacore Tuna ...
9,B0005XMTHE,Smucker's,$2.29,"[Helps build muscles with bcaa's amino acids, ..."


Next let's drop duplicate rows based on the `asin` column value. There should only be one duplicate based on the `describe()` output above.

In [14]:
items_df = items_df.drop_duplicates(subset=['asin'], keep='last')
items_df.shape

(10812, 4)

Next let's focus on reformatting and cleaning up the `description` column values. As you can see above, the `description` is currently represented as an array of strings (because that's how it is represented in the JSON file). We need to flatten this array into a single string and strip all HTML markup from each fragment.

We'll start by creating two utility functions that will be used to clean the `description` (and later the `title` column in the original dataset when we want to display titles for recommended products).

In [15]:
# Strips and cleans a value of HTML markup and whitespace.
def clean_markup(value):
    s = str(value).strip()
    if s != '':
        s = str(html.fromstring(s).text_content())
        s = ' '.join(s.split())
                
    return s.strip()

# Cleans and reformats the description column value for a dataframe row.
def clean_and_reformat_description(row):
    s = ''
    for el in row['description']:
        el = clean_markup(el)
        if el != '':
            s += ' ' + el
                
    return s.strip()

In [16]:
items_df['description'] = items_df.apply(clean_and_reformat_description, axis=1)
items_df

,asin,brand,price,description
1,B00005BPJO,Milk Duds,$5.00,Sink your sweet tooth into MILK DUDS Candya de...
2,B0000DIF38,Goya,,A perfect Lentil soup starts with Goya Lentils...
3,B0000DIWNI,Saran,,Saran Premium Wrap is an extra tough yet easy ...
4,B0000DIWNZ,Saran,,200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Easy...
5,B0000GH6UG,Ibarra,,"Ibarra Chocolate, 19 Oz"
...,...,...,...,...
10808,B01HI76312,KIND,$3.98,These bars are where our journey started and i...
10809,B01HI76790,KIND,$5.81,These bars are where our journey started and i...
10810,B01HI76SA8,KIND,$4.98,These bars are where our journey started and i...
10811,B01HI76XS0,KIND,$5.81,These bars are where our journey started and i...


Next let's take a look at the `price` column and change its type from a string to a float.

In [17]:
items_df['price'].value_counts()

          4063
$2.99      114
$3.99      113
$4.99      103
$5.99       87
          ... 
$20.42       1
$32.32       1
$1.52        1
$27.89       1
$39.10       1
Name: price, Length: 1482, dtype: int64

The following cell with convert empty/non-numeric prices to `np.nan` and all others will have the `$` currency symbol removed. This will allow us to coerce the type to a float.

In [18]:
def convert_price(row):
    v = str(row['price']).strip().replace('$', '')
    if v == '' or not v.lstrip('-').replace('.', '').isdigit():
        return np.nan
    return v

items_df['price'] = items_df.apply(convert_price, axis=1)
items_df

,asin,brand,price,description
1,B00005BPJO,Milk Duds,5.00,Sink your sweet tooth into MILK DUDS Candya de...
2,B0000DIF38,Goya,NaN,A perfect Lentil soup starts with Goya Lentils...
3,B0000DIWNI,Saran,NaN,Saran Premium Wrap is an extra tough yet easy ...
4,B0000DIWNZ,Saran,NaN,200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Easy...
5,B0000GH6UG,Ibarra,NaN,"Ibarra Chocolate, 19 Oz"
...,...,...,...,...
10808,B01HI76312,KIND,3.98,These bars are where our journey started and i...
10809,B01HI76790,KIND,5.81,These bars are where our journey started and i...
10810,B01HI76SA8,KIND,4.98,These bars are where our journey started and i...
10811,B01HI76XS0,KIND,5.81,These bars are where our journey started and i...


In [19]:
items_df['price'].value_counts()

2.99     114
3.99     113
4.99     103
5.99      87
2.98      76
        ... 
39.10      1
1.84       1
22.95      1
12.17      1
11.09      1
Name: price, Length: 1480, dtype: int64

In [20]:
items_df['price'] = items_df['price'].astype(float)

Next we'll rename the columns to match the names and uppercase name format expected by Personalize.

In [21]:
items_df.rename(columns = {'asin':'ITEM_ID', 'brand':'BRAND', 
                              'price':'PRICE', 'description': 'DESCRIPTION'}, inplace = True)
items_df.head(10)

,ITEM_ID,BRAND,PRICE,DESCRIPTION
1,B00005BPJO,Milk Duds,5.00,Sink your sweet tooth into MILK DUDS Candya de...
2,B0000DIF38,Goya,NaN,A perfect Lentil soup starts with Goya Lentils...
3,B0000DIWNI,Saran,NaN,Saran Premium Wrap is an extra tough yet easy ...
4,B0000DIWNZ,Saran,NaN,200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Easy...
5,B0000GH6UG,Ibarra,NaN,"Ibarra Chocolate, 19 Oz"
6,B0000KC2BK,Knorr,3.09,Knorr Granulated Chicken Flavor Bouillon is a ...
7,B0001E1IN8,Castillo,NaN,Red chili habanero sauces. They are present to...
8,B00032E8XK,Chicken of the Sea,1.48,Chicken of the Sea Solid White Albacore Tuna i...
9,B0005XMTHE,Smucker's,2.29,"Helps build muscles with bcaa's amino acids, i..."
10,B0005XNE6E,Snapple,1.99,"At Snapple, we believe lifes a peach. Weve bee..."


We'll be creating two items CSVs. One will have the description column and the other will not. We'll use each of these to train separate models with the same recipe so we can compare the offline metrics and do some online inspection of recommendations.

In [22]:
items_with_desc_filename = "items-with-desc.csv"
items_df.to_csv(items_with_desc_filename, index=False, float_format='%.2f')

Another items CSV with the description column removed.

In [23]:
items_without_desc_df = items_df[['ITEM_ID', 'BRAND', 'PRICE']]
items_without_desc_df.head()

,ITEM_ID,BRAND,PRICE
1,B00005BPJO,Milk Duds,5.0
2,B0000DIF38,Goya,NaN
3,B0000DIWNI,Saran,NaN
4,B0000DIWNZ,Saran,NaN
5,B0000GH6UG,Ibarra,NaN


In [24]:
items_without_desc_filename = "items-without-desc.csv"
items_without_desc_df.to_csv(items_without_desc_filename, index=False, float_format='%.2f')

## Create dataset groups and upload datasets

With the datasets that we need built, now it's time to upload them to Personalize using dataset import jobs. Before we can upload the CSVs, we need to create dataset groups to hold our two dataset approaches (without and with descriptions), create schemas for our datasets, and create datasets.

We'll start by creating SDK client that we'll need to interact with Personalize.

In [25]:
personalize = boto3.client('personalize')

### Create dataset groups

Let's create our two dataset groups.

In [26]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "amazon-pantry-without-desc"
)

dataset_group_without_desc_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:224124347618:dataset-group/amazon-pantry-without-desc",
  "ResponseMetadata": {
    "RequestId": "20bd153c-ebd0-432d-9ef3-522a0d2fd8d4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:08:18 GMT",
      "x-amzn-requestid": "20bd153c-ebd0-432d-9ef3-522a0d2fd8d4",
      "content-length": "105",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [27]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "amazon-pantry-with-desc"
)

dataset_group_with_desc_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:224124347618:dataset-group/amazon-pantry-with-desc",
  "ResponseMetadata": {
    "RequestId": "9cec53c8-28a3-40e5-bf0e-6f87a03113f7",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:08:18 GMT",
      "x-amzn-requestid": "9cec53c8-28a3-40e5-bf0e-6f87a03113f7",
      "content-length": "102",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Since dataset groups can take a few seconds to be fully created, let's wait until they both have a status of ACTIVE.

In [28]:
in_progress_dataset_group_arns = [ dataset_group_without_desc_arn, dataset_group_with_desc_arn ]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for dataset_group_arn in in_progress_dataset_group_arns:
        describe_dataset_group_response = personalize.describe_dataset_group(
            datasetGroupArn = dataset_group_arn
        )
        status = describe_dataset_group_response["datasetGroup"]["status"]
        if status == "ACTIVE":
            print("Dataset group create succeeded for {}".format(dataset_group_arn))
            in_progress_dataset_group_arns.remove(dataset_group_arn)
        elif status == "CREATE FAILED":
            print("Create failed for {}".format(dataset_group_arn))
            in_progress_dataset_group_arns.remove(dataset_group_arn)

    if len(in_progress_dataset_group_arns) <= 0:
        break
    else:
        print("At least one dataset group create is still in progress")
                
    time.sleep(10)

At least one dataset group create is still in progress
Dataset group create succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-group/amazon-pantry-without-desc
At least one dataset group create is still in progress
Dataset group create succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-group/amazon-pantry-with-desc


### Create Interactions dataset schema and datasets

Since the interactions dataset will be the same for both dataset groups, we will be creating a single schema for the interactions dataset type and sharing it across both dataset groups. This is possible since schemas are global to your AWS account and not specific to a dataset group.

In [29]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "EVENT_VALUE",
            "type": "float"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
        }
    ],
    "version": "1.0"
}
            
create_schema_response = personalize.create_schema(
    name = "amazon-pantry-interactions",
    schema = json.dumps(interactions_schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:224124347618:schema/amazon-pantry-interactions",
  "ResponseMetadata": {
    "RequestId": "6be5e019-0c8e-487b-a158-6f089f9e79ca",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:08:40 GMT",
      "x-amzn-requestid": "6be5e019-0c8e-487b-a158-6f089f9e79ca",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Next we'll create an Interactions dataset in both dataset groups specifying the schema we just created.

In [30]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "amazon-pantry-without-desc-ints",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_without_desc_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_without_desc_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:224124347618:dataset/amazon-pantry-without-desc/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "ea1020bb-a7fa-4a64-a29e-f07e8e0f9ae9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:08:41 GMT",
      "x-amzn-requestid": "ea1020bb-a7fa-4a64-a29e-f07e8e0f9ae9",
      "content-length": "107",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [31]:
create_dataset_response = personalize.create_dataset(
    name = "amazon-pantry-with-desc-ints",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_with_desc_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_with_desc_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:224124347618:dataset/amazon-pantry-with-desc/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "614257cc-6344-408e-8e15-89d4f81ae927",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:08:41 GMT",
      "x-amzn-requestid": "614257cc-6344-408e-8e15-89d4f81ae927",
      "content-length": "104",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Stage Interactions CSV in S3

Before we can upload the interactions CSV we created earlier into the Personalize datasets that we just created, we need to stage the CSV in an S3 bucket.

Let's create an S3 bucket and copy the interactions CSV file to the bucket.

In [32]:
# Determine the current S3 region where this notebook is being hosted in SageMaker.
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

us-east-1


In [33]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "amazon-pantry-personalize-text"
print(bucket_name)
if region == "us-east-1":
    s3.create_bucket(Bucket=bucket_name)
else:
    s3.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={'LocationConstraint': region}
    )

224124347618-us-east-1-amazon-pantry-personalize-text


#### Upload Interactions CSV to S3

In [34]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_filename)

### Create S3 bucket policy and IAM role

Before we can submit a dataset import job to Personalize, we have to create a bucket policy and IAM role that will give Personalize access to our bucket.

In [35]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': 'SBN10P9R7H7ST5RK',
  'HostId': 'DD8fYEx27yBq6/rB7o9lMvkdCLOHOewN05NSq73g30jeFBdouLj5D+fWSnIZHvDuAKdCKEo7w3k=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'DD8fYEx27yBq6/rB7o9lMvkdCLOHOewN05NSq73g30jeFBdouLj5D+fWSnIZHvDuAKdCKEo7w3k=',
   'x-amz-request-id': 'SBN10P9R7H7ST5RK',
   'date': 'Tue, 13 Jul 2021 22:10:59 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [36]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleAmazonPantry"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(20) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::224124347618:role/PersonalizeRoleAmazonPantry


### Import Interactions datasets for each dataset group

Now we're ready to import the staged Interactions CSV in our S3 bucket to the Personalize datasets we created in each dataset group. We'll submit both import jobs and wait for them both to complete.

In [37]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "amazon-pantry-without-desc-ints-import",
    datasetArn = interactions_dataset_without_desc_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_without_ints_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-without-desc-ints-import",
  "ResponseMetadata": {
    "RequestId": "84c4d71d-fe71-4ee7-bccf-4f8ed8b1e549",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:12:08 GMT",
      "x-amzn-requestid": "84c4d71d-fe71-4ee7-bccf-4f8ed8b1e549",
      "content-length": "126",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [38]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "amazon-pantry-with-desc-ints-import",
    datasetArn = interactions_dataset_with_desc_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_with_ints_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-with-desc-ints-import",
  "ResponseMetadata": {
    "RequestId": "eae39716-264b-48e8-bfb4-93f569c7a904",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:12:09 GMT",
      "x-amzn-requestid": "eae39716-264b-48e8-bfb4-93f569c7a904",
      "content-length": "123",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Wait for Interactions dataset import jobs to complete

The following cell will wait for both import jobs to complete.

In [39]:
%%time

in_progress_import_arns = [ dataset_import_job_without_ints_arn, dataset_import_job_with_ints_arn ]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for import_arn in in_progress_import_arns:
        describe_dataset_import_job_response = personalize.describe_dataset_import_job(
            datasetImportJobArn = import_arn
        )
        status = describe_dataset_import_job_response["datasetImportJob"]['status']
        if status == "ACTIVE":
            print("Dataset import succeeded for {}".format(import_arn))
            in_progress_import_arns.remove(import_arn)
        elif status == "CREATE FAILED":
            print("Create failed for {}".format(import_arn))
            in_progress_import_arns.remove(import_arn)

    if len(in_progress_import_arns) <= 0:
        break
    else:
        print("At least one dataset import job is still in progress")
                
    time.sleep(60)

At least one dataset import job is still in progress
At least one dataset import job is still in progress
At least one dataset import job is still in progress
At least one dataset import job is still in progress
Dataset import succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-without-desc-ints-import
At least one dataset import job is still in progress
Dataset import succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-with-desc-ints-import
CPU times: user 42.3 ms, sys: 6.1 ms, total: 48.4 ms
Wall time: 5min


### Create Items dataset schema and datasets

Next we will repeat the process for the items datasets. This time, though, we will need to create two schemas since one items dataset includes the description column and the other does not. We'll start with the schema that does not include the description.

In [40]:
item_without_desc_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "BRAND",
            "type": [ "null", "string" ],
            "categorical": True
        },{
            "name": "PRICE",
            "type": [ "null", "float" ],
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "amazon-pantry-item-without-desc-schema",
    schema = json.dumps(item_without_desc_schema)
)

item_without_desc_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:224124347618:schema/amazon-pantry-item-without-desc-schema",
  "ResponseMetadata": {
    "RequestId": "11a18b05-189b-4485-ba8b-58083e784321",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:17:16 GMT",
      "x-amzn-requestid": "11a18b05-189b-4485-ba8b-58083e784321",
      "content-length": "104",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Next we'll create a schema that includes the description. Be sure to take note of the `"textual": True` attribute on the `DESCRIPTION` field. This is how you differentiate unstructured text fields from categorical and string fields. Without this attribute, Personalize will not apply natural language processing techniques to extract features from this text.

In [41]:
item_with_desc_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "BRAND",
            "type": [ "null", "string" ],
            "categorical": True
        },{
            "name": "PRICE",
            "type": [ "null", "float" ],
        },{
            "name": "DESCRIPTION",
            "type": [ "null", "string" ],
            "textual": True
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "amazon-pantry-item-with-desc-schema",
    schema = json.dumps(item_with_desc_schema)
)

item_with_desc_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:224124347618:schema/amazon-pantry-item-with-desc-schema",
  "ResponseMetadata": {
    "RequestId": "05c60e80-1d7c-45f1-a881-d08af62a8432",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:17:16 GMT",
      "x-amzn-requestid": "05c60e80-1d7c-45f1-a881-d08af62a8432",
      "content-length": "101",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Next we will create Personalize datasets in each dataset group, taking special care to specify the approprate schema ARN for each dataset.

In [42]:
dataset_type = "ITEMS"
create_dataset_response = personalize.create_dataset(
    name = "amazon-pantry-without-desc-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_without_desc_arn,
    schemaArn = item_without_desc_schema_arn
)

items_dataset_without_desc_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:224124347618:dataset/amazon-pantry-without-desc/ITEMS",
  "ResponseMetadata": {
    "RequestId": "f9b87f6c-22c8-42a7-8a3a-8c203300e3eb",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:18:13 GMT",
      "x-amzn-requestid": "f9b87f6c-22c8-42a7-8a3a-8c203300e3eb",
      "content-length": "100",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [43]:
create_dataset_response = personalize.create_dataset(
    name = "amazon-pantry-with-desc-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_with_desc_arn,
    schemaArn = item_with_desc_schema_arn
)

items_dataset_with_desc_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:224124347618:dataset/amazon-pantry-with-desc/ITEMS",
  "ResponseMetadata": {
    "RequestId": "cbba4d2a-17c9-4a9b-a6a3-a2df934e2de1",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:18:16 GMT",
      "x-amzn-requestid": "cbba4d2a-17c9-4a9b-a6a3-a2df934e2de1",
      "content-length": "97",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Stage Items CSV in S3

Next we'll copy our two items CSV files to the same S3 bucket create above.

In [44]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(items_without_desc_filename).upload_file(items_without_desc_filename)

In [45]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(items_with_desc_filename).upload_file(items_with_desc_filename)

### Import Items datasets for each dataset group

Since the S3 bucket policy and IAM role are already setup, we can just submit two dataset import jobs to import the Items CSVs.

In [46]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "amazon-pantry-without-desc-items-import",
    datasetArn = items_dataset_without_desc_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, items_without_desc_filename)
    },
    roleArn = role_arn
)

dataset_import_job_without_items_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-without-desc-items-import",
  "ResponseMetadata": {
    "RequestId": "c6ea207b-b8eb-4565-8ce2-6eb44e0fa18f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:18:37 GMT",
      "x-amzn-requestid": "c6ea207b-b8eb-4565-8ce2-6eb44e0fa18f",
      "content-length": "127",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [47]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "amazon-pantry-with-desc-items-import",
    datasetArn = items_dataset_with_desc_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, items_with_desc_filename)
    },
    roleArn = role_arn
)

dataset_import_job_with_items_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-with-desc-items-import",
  "ResponseMetadata": {
    "RequestId": "cf95206f-1527-4247-a618-6c8c832fa05f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:18:38 GMT",
      "x-amzn-requestid": "cf95206f-1527-4247-a618-6c8c832fa05f",
      "content-length": "124",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Wait for Items import jobs to complete

The following logic will wait until both items datasets are fully imported into each dataset group.

In [48]:
%%time

in_progress_import_arns = [ dataset_import_job_without_items_arn, dataset_import_job_with_items_arn ]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for import_arn in in_progress_import_arns:
        describe_dataset_import_job_response = personalize.describe_dataset_import_job(
            datasetImportJobArn = import_arn
        )
        status = describe_dataset_import_job_response["datasetImportJob"]['status']
        if status == "ACTIVE":
            print("Dataset import succeeded for {}".format(import_arn))
            in_progress_import_arns.remove(import_arn)
        elif status == "CREATE FAILED":
            print("Create failed for {}".format(import_arn))
            in_progress_import_arns.remove(import_arn)

    if len(in_progress_import_arns) <= 0:
        break
    else:
        print("At least one dataset import job is still in progress")
                
    time.sleep(60)

At least one dataset import job is still in progress
At least one dataset import job is still in progress
At least one dataset import job is still in progress
At least one dataset import job is still in progress
Dataset import succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-with-desc-items-import
At least one dataset import job is still in progress
At least one dataset import job is still in progress
At least one dataset import job is still in progress
Dataset import succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-without-desc-items-import
CPU times: user 57.6 ms, sys: 5.06 ms, total: 62.7 ms
Wall time: 7min


## Create solutions and solution versions

With the interactions and items datasets imported into each dataset group, we will next create solutions and solution versions using the user-personalization recipe for the data in each dataset group.

First, let's list the Personalize recipes available.

In [49]:
personalize.list_recipes()

{'recipes': [{'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2021, 2, 6, 19, 6, 40, 447000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2021, 2, 6, 19, 6, 40, 447000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-metadata',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-metadata',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2021, 2, 6, 19, 6, 40, 447000, tzinfo=tzlocal())},
  {'name': 'aws-personalized-ranking',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-personalized-ranking',
   'stat

We will use the user-personalization recipe for this notebook since it is one of the recipes that uses item metadata. This recipe supports the canonical personalization use case where given a user, you want Personalize to recommend items that this user would be interested in. 

In [50]:
user_personalization_recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

First, we will create a solution and solution version in the dataset group that does not include item descriptions.

In [51]:
user_personalization_create_solution_response = personalize.create_solution(
    name = "amazon-pantry-without-desc-userpersonalization",
    datasetGroupArn = dataset_group_without_desc_arn,
    recipeArn = user_personalization_recipe_arn
)

user_personalization_without_desc_solution_arn = user_personalization_create_solution_response['solutionArn']

In [52]:
print(user_personalization_without_desc_solution_arn)

arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-without-desc-userpersonalization


In [53]:
user_personalization_solution_version_response = personalize.create_solution_version(
    solutionArn = user_personalization_without_desc_solution_arn
)

In [54]:
user_personalization_without_solution_version_arn = user_personalization_solution_version_response['solutionVersionArn']
print(json.dumps(user_personalization_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-without-desc-userpersonalization/0b76212f",
  "ResponseMetadata": {
    "RequestId": "018b3fcb-10d5-4290-a17a-3970723abacd",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:26:14 GMT",
      "x-amzn-requestid": "018b3fcb-10d5-4290-a17a-3970723abacd",
      "content-length": "132",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Next we will create a solution and solution version in the dataset group that includes the item descriptions.

In [55]:
user_personalization_create_solution_response = personalize.create_solution(
    name = "amazon-pantry-with-desc-userpersonalization",
    datasetGroupArn = dataset_group_with_desc_arn,
    recipeArn = user_personalization_recipe_arn
)

user_personalization_with_desc_solution_arn = user_personalization_create_solution_response['solutionArn']

In [56]:
print(user_personalization_with_desc_solution_arn)

arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-with-desc-userpersonalization


In [57]:
user_personalization_solution_version_response = personalize.create_solution_version(
    solutionArn = user_personalization_with_desc_solution_arn
)

In [58]:
user_personalization_with_solution_version_arn = user_personalization_solution_version_response['solutionVersionArn']
print(json.dumps(user_personalization_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-with-desc-userpersonalization/f178990f",
  "ResponseMetadata": {
    "RequestId": "f630b862-6fa9-4eb7-a0d2-71d0b9637e80",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:26:29 GMT",
      "x-amzn-requestid": "f630b862-6fa9-4eb7-a0d2-71d0b9637e80",
      "content-length": "129",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Wait for solution versions to become active

Finally, we'll wait for the solution versions to finish being created. This step is where Personalize trains machine learning models based on the datasets and selected recipe. Personalize will also split the interactions datasets into training and evaluation portions so it can evaluate the quality of recommendations against the trained model using held out data.

You will notice that the solution version in the dataset group that includes the description data will take longer to train than the one without the description.

In [59]:
%%time

in_progress_solution_versions = [
    user_personalization_without_solution_version_arn,
    user_personalization_with_solution_version_arn
]

max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:
    for solution_version_arn in in_progress_solution_versions:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
    
    if len(in_progress_solution_versions) <= 0:
        break
    else:
        print("At least one solution build is still in progress")
        
    time.sleep(60)

At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solutio

Generally speaking, the addition of text-based unstructured meta will increase training time. In our case, you can see above that the solution version that trained on the dataset with product descriptions took about 15 minutes longer than the solution version trained on the dataset without production descriptions. This difference will vary based on the composition and text values for your datasets.

Let's inspect the training hours for each solution version and compare them as well.

In [60]:
response = personalize.describe_solution_version(solutionVersionArn = user_personalization_without_solution_version_arn)
training_hours_without_desc = response['solutionVersion']['trainingHours']

response = personalize.describe_solution_version(solutionVersionArn = user_personalization_with_solution_version_arn)
training_hours_with_desc = response['solutionVersion']['trainingHours']
training_diff = (training_hours_with_desc - training_hours_without_desc) / training_hours_without_desc

print(f"Training hours without description: {training_hours_without_desc}")
print(f"Training hours with description: {training_hours_with_desc}")

print("Difference of {:.2%}".format(training_diff))

Training hours without description: 4.199
Training hours with description: 5.346
Difference of 27.32%


The training hours used for cost calculations was about 27% higher for training with the description column. 

The wall/clock time and training hours will vary depending on the size of your datasets but this information can help you assess the trade off when considering adding unstructured text to your datasets.

### Inspect offline metrics

Now that the solution versions have finished building, let's inspect and compare the offline metrics for each solution version to see how the inclusion of unstructured text has impacted these metrics.

In [61]:
metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = user_personalization_without_solution_version_arn
)

print(json.dumps(metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-without-desc-userpersonalization/0b76212f",
  "metrics": {
    "coverage": 0.0914,
    "mean_reciprocal_rank_at_25": 0.0268,
    "normalized_discounted_cumulative_gain_at_10": 0.0376,
    "normalized_discounted_cumulative_gain_at_25": 0.0464,
    "normalized_discounted_cumulative_gain_at_5": 0.0309,
    "precision_at_10": 0.0058,
    "precision_at_25": 0.0037,
    "precision_at_5": 0.0076
  },
  "ResponseMetadata": {
    "RequestId": "8c61339a-f929-47e0-81f0-a9660ebd589f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 23:16:07 GMT",
      "x-amzn-requestid": "8c61339a-f929-47e0-81f0-a9660ebd589f",
      "content-length": "430",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Let's save these off in a dictionary so we can more easily compare the metrics between both solution versions.

In [62]:
metrics = {
    'Coverage': [ metrics_response['metrics']['coverage'] ],
    'MRR-25': [ metrics_response['metrics']['mean_reciprocal_rank_at_25'] ],
    'NDCG-5': [ metrics_response['metrics']['normalized_discounted_cumulative_gain_at_5'] ],
    'NDCG-10': [ metrics_response['metrics']['normalized_discounted_cumulative_gain_at_10'] ],
    'NDCG-25': [ metrics_response['metrics']['normalized_discounted_cumulative_gain_at_25'] ],    
    'Precision-5': [ metrics_response['metrics']['precision_at_5'] ],
    'Precision-10': [ metrics_response['metrics']['precision_at_10'] ],
    'Precision-25': [ metrics_response['metrics']['precision_at_25'] ],    
}

Next, get the offline metrics for the solution version that included the description column and save them off too.

In [63]:
metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = user_personalization_with_solution_version_arn
)

print(json.dumps(metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-with-desc-userpersonalization/f178990f",
  "metrics": {
    "coverage": 0.1323,
    "mean_reciprocal_rank_at_25": 0.0367,
    "normalized_discounted_cumulative_gain_at_10": 0.049,
    "normalized_discounted_cumulative_gain_at_25": 0.0591,
    "normalized_discounted_cumulative_gain_at_5": 0.0425,
    "precision_at_10": 0.0071,
    "precision_at_25": 0.0045,
    "precision_at_5": 0.0104
  },
  "ResponseMetadata": {
    "RequestId": "b54df693-4378-4194-96c7-cec3a9d934cf",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 23:16:14 GMT",
      "x-amzn-requestid": "b54df693-4378-4194-96c7-cec3a9d934cf",
      "content-length": "426",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [64]:
metrics['Coverage'].append(metrics_response['metrics']['coverage'])
metrics['MRR-25'].append(metrics_response['metrics']['mean_reciprocal_rank_at_25'])
metrics['NDCG-5'].append(metrics_response['metrics']['normalized_discounted_cumulative_gain_at_5'])
metrics['NDCG-10'].append(metrics_response['metrics']['normalized_discounted_cumulative_gain_at_10'])
metrics['NDCG-25'].append(metrics_response['metrics']['normalized_discounted_cumulative_gain_at_25'])
metrics['Precision-5'].append(metrics_response['metrics']['precision_at_5'])
metrics['Precision-10'].append(metrics_response['metrics']['precision_at_10'])
metrics['Precision-25'].append(metrics_response['metrics']['precision_at_25'])

Calculate the percentage change in each metric for with and without text and display the results.

In [65]:
for key in metrics:
    metrics[key].append("{:.2%}".format((metrics[key][1] - metrics[key][0])/metrics[key][0]))

metrics_df = pd.DataFrame.from_dict(metrics,orient='index',columns=['Without Text', 'With Text', '% Change'])
metrics_df

,Without Text,With Text,% Change
Coverage,0.0914,0.1323,44.75%
MRR-25,0.0268,0.0367,36.94%
NDCG-5,0.0309,0.0425,37.54%
NDCG-10,0.0376,0.0490,30.32%
NDCG-25,0.0464,0.0591,27.37%
Precision-5,0.0076,0.0104,36.84%
Precision-10,0.0058,0.0071,22.41%
Precision-25,0.0037,0.0045,21.62%


These metrics clearly show that recommendations from the solution version that includes the item descriptions were significantly better across the board. For more sparse interactions datasets where users and items have fewer interactions will benefit more from the addition of text than datasets that already have a higher number of interactions per item and/or user.

## Cleanup

Personalize resources created by this notebook can be deleted from the Personalize service page in the AWS console. 

Alternatively, the following script can be run locally to delete all resources in each dataset group.

https://gist.github.com/james-jory/62ddddf2f9180b77dd2a42e645b9d3b0

In addition, the IAM Role and S3 bucket can be deleted from the IAM and S3 service pages, respectively.